In [6]:
import os

In [7]:
%pwd

'/Users/main/Desktop/LSTM_Forecast/research'

In [10]:
os.chdir('../')

In [11]:
%pwd

'/Users/main/Desktop/LSTM_Forecast'

In [13]:
import pandas as pd

data= pd.read_csv('./artifacts/data_ingestion/apple.csv')

In [14]:
data.columns

Index(['Date', 'Close/Last', 'Volume', 'Open', 'High', 'Low'], dtype='object')

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [16]:
from TimeSeriesForecast.constants import *
from TimeSeriesForecast.utils.common import read_yaml,create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH,
        schema_filepath= SCHEMA_FILE_PATH):
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self)-> DataValidationConfig:
        config= self.config.data_validation
        schema= self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config= DataValidationConfig(
            root_dir=  config.root_dir,
            STATUS_FILE= config.STATUS_FILE,
            unzip_data_dir= config.unzip_data_dir,
            all_schema= schema
        )
        
        return data_validation_config

In [18]:
import os
from TimeSeriesForecast import logger

In [19]:
class DataValidation:
    def __init__(self,config: DataValidationConfig):
        self.config= config
    
    def validate_all_columns(self)->bool:
        try:
            validation_status = None
            data= pd.read_csv(self.config.unzip_data_dir)
            all_columns= list(data.columns)
            
            all_schema= self.config.all_schema.keys()
            
            for col in all_columns:
                if col not in all_schema:
                    validation_status= False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE,'w') as x:
                        x.write(f"Validation status: {validation_status}")
            return validation_status
        except Exception as e:
            raise e

In [20]:
try:
    config= ConfigurationManager()
    data_validation_config= config.get_data_validation_config()
    data_validation= DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-07-18 18:24:11,361:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-07-18 18:24:11,363:INFO:common:yaml file: params.yaml loaded successfully]
[2024-07-18 18:24:11,366:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-07-18 18:24:11,367:INFO:common:created directory at: artifacts]
[2024-07-18 18:24:11,367:INFO:common:created directory at: artifacts/data_validation]
